Install the following packages before running the notebook

In [3]:
%%bash
pip install --upgrade tensorflow==1.4
pip install --ignore-installed --upgrade pytz==2018.4
pip uninstall -y google-cloud-dataflow
pip install --upgrade apache-beam[gcp]==2.12.0
pip install google-cloud-bigquery --upgrade

  Using cached https://files.pythonhosted.org/packages/99/72/a420e22dc93416d30981e87a2318823ec09a9b18631369df0e7d9d164073/tensorflow-1.4.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/64/cd/f3d14d441eb1c5228aaf7e12e8e94895ae73e9af50383e481610b34357bd/tensorflow_tensorboard-0.4.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/33/70/86c5fec937ea4964184d4d6c4f0b9551564f821e1c3575907639036d9b90/bleach-1.5.0-py2.py3-none-any.whl
  Found existing installation: html5lib 1.1
    Uninstalling html5lib-1.1:
      Successfully uninstalled html5lib-1.1
  Found existing installation: bleach 2.1.2
    Uninstalling bleach-2.1.2:
      Successfully uninstalled bleach-2.1.2
  Found existing installation: tensorflow 1.8.0
    Uninstalling tensorflow-1.8.0:
      Successfully uninstalled tensorflow-1.8.0
  Using cached https://files.pythonhosted.org/packages/dc/83/15f7833b70d3e067ca91467ca245bae0f6fe56ddc7451aa0dc5606b120f2/pytz-2018.4-

apache-airflow 1.9.0 has requirement bleach==2.1.2, but you'll have bleach 1.5.0 which is incompatible.
google-cloud-storage 1.36.1 has requirement google-cloud-core<2.0dev,>=1.4.1, but you'll have google-cloud-core 0.28.1 which is incompatible.
google-cloud-storage 1.36.1 has requirement google-resumable-media<2.0dev,>=1.2.0, but you'll have google-resumable-media 0.4.1 which is incompatible.
apache-airflow 1.9.0 has requirement bleach==2.1.2, but you'll have bleach 1.5.0 which is incompatible.
googledatastore 7.0.1 has requirement httplib2<0.10,>=0.9.1, but you'll have httplib2 0.11.3 which is incompatible.
google-cloud-bigtable 0.31.1 has requirement google-cloud-core<0.29dev,>=0.28.0, but you'll have google-cloud-core 1.6.0 which is incompatible.
google-cloud-core 1.6.0 has requirement google-auth<2.0dev,>=1.24.0, but you'll have google-auth 1.21.1 which is incompatible.
apache-airflow 1.9.0 has requirement bleach==2.1.2, but you'll have bleach 1.5.0 which is incompatible.


Create DataSet on BigQuery

In [6]:
%%bash
bq mk --dataset sevir-306302:sevirdataset

Dataset 'sevir-306302:sevirdataset' successfully created.


Creating Tables on BigQuery

In [61]:
%%bash
bq mk -t sevirdataset.catalogcsv \
id:STRING,file_name:STRING,file_index:INTEGER,img_type:STRING,time_utc:DATETIME,episode_id:STRING,event_id:STRING,event_type:STRING,llcrnrlat:FLOAT,llcrnrlon:FLOAT,urcrnrlat:FLOAT,urcrnrlon:FLOAT,proj:STRING,size_x:INTEGER,size_y:INTEGER,height_m:FLOAT,width_m:FLOAT,data_min:FLOAT,data_max:FLOAT,pct_missing:FLOAT
bq mk -t sevirdataset.storm_details \
BEGIN_YEARMONTH:INTEGER,BEGIN_DAY:INTEGER,BEGIN_TIME:INTEGER,END_YEARMONTH:INTEGER,END_DAY:INTEGER,END_TIME:INTEGER,EPISODE_ID:STRING,EVENT_ID:INTEGER,STATE:STRING,STATE_FIPS:INTEGER,YEAR:INTEGER,MONTH_NAME:STRING,EVENT_TYPE:STRING,CZ_TYPE:STRING,CZ_FIPS:INTEGER,CZ_NAME:STRING,WFO:STRING,BEGIN_DATE_TIME:STRING,CZ_TIMEZONE:STRING,END_DATE_TIME:STRING,INJURIES_DIRECT:STRING,INJURIES_INDIRECT:STRING,DEATHS_DIRECT:STRING,DEATHS_INDIRECT:STRING,DAMAGE_PROPERTY:STRING,DAMAGE_CROPS:STRING,SOURCE:STRING,MAGNITUDE:STRING,MAGNITUDE_TYPE:STRING,FLOOD_CAUSE:STRING,CATEGORY:STRING,BEGIN_RANGE:STRING,BEGIN_AZIMUTH:STRING,BEGIN_LOCATION:STRING,END_RANGE:STRING,END_AZIMUTH:STRING,END_LOCATION:STRING,BEGIN_LAT:STRING,BEGIN_LON:STRING,END_LAT:STRING,END_LON:STRING,EPISODE_NARRATIVE:STRING,EVENT_NARRATIVE:STRING,DATA_SOURCE:STRING

Table 'sevir-306302:sevirdataset.catalogcsv' successfully created.
Table 'sevir-306302:sevirdataset.storm_details' successfully created.


Pipeline to move data from cloud Storage to Bigquery

In [63]:
import apache_beam as beam
import argparse
from apache_beam.options.pipeline_options import PipelineOptions
from sys import argv
PROJECT_ID = 'sevir-306302'
SCHEMACATALOG = 'id:STRING,file_name:STRING,file_index:INTEGER,img_type:STRING,time_utc:DATETIME,episode_id:STRING,event_id:STRING,event_type:STRING,llcrnrlat:FLOAT,llcrnrlon:FLOAT,urcrnrlat:FLOAT,urcrnrlon:FLOAT,proj:STRING,size_x:INTEGER,size_y:INTEGER,height_m:FLOAT,width_m:FLOAT,data_min:FLOAT,data_max:FLOAT,pct_missing:FLOAT'
SCHEMASTORM = 'BEGIN_YEARMONTH:INTEGER,BEGIN_DAY:INTEGER,BEGIN_TIME:INTEGER,END_YEARMONTH:INTEGER,END_DAY:INTEGER,END_TIME:INTEGER,EPISODE_ID:STRING,EVENT_ID:INTEGER,STATE:STRING,STATE_FIPS:INTEGER,YEAR:INTEGER,MONTH_NAME:STRING,EVENT_TYPE:STRING,CZ_TYPE:STRING,CZ_FIPS:INTEGER,CZ_NAME:STRING,WFO:STRING,BEGIN_DATE_TIME:STRING,CZ_TIMEZONE:STRING,END_DATE_TIME:STRING,INJURIES_DIRECT:STRING,INJURIES_INDIRECT:STRING,DEATHS_DIRECT:STRING,DEATHS_INDIRECT:STRING,DAMAGE_PROPERTY:STRING,DAMAGE_CROPS:STRING,SOURCE:STRING,MAGNITUDE:STRING,MAGNITUDE_TYPE:STRING,FLOOD_CAUSE:STRING,CATEGORY:STRING,BEGIN_RANGE:STRING,BEGIN_AZIMUTH:STRING,BEGIN_LOCATION:STRING,END_RANGE:STRING,END_AZIMUTH:STRING,END_LOCATION:STRING,BEGIN_LAT:STRING,BEGIN_LON:STRING,END_LAT:STRING,END_LON:STRING,EPISODE_NARRATIVE:STRING,EVENT_NARRATIVE:STRING,DATA_SOURCE:STRING'


def dropcolums(data):
    del data['minute_offsets']
    return data

def convert_types(data):
    """Converts string values to their appropriate type."""
    data['height_m'] = float(data['height_m']) if 'height_m' in data else None
    data['width_m'] = float(data['width_m']) if 'width_m' in data else None
    return data

def dropcolumsstorm(data):
    del data['TOR_F_SCALE'],
    del data['TOR_LENGTH'],
    del data['TOR_WIDTH'],
    del data['TOR_OTHER_WFO'],
    del data['TOR_OTHER_CZ_STATE'],
    del data['TOR_OTHER_CZ_FIPS'],
    del data['TOR_OTHER_CZ_NAME']
    return data

  
p = beam.Pipeline(options=PipelineOptions(flags=argv,
    runner='DataflowRunner',
    project='sevir-306302',
    job_name='catalog',
    temp_location='gs://sevir-306302/temp',
    staging_location='gs://sevir-306302/tempstg',
    region='us-central1'))
(p | 'ReadDataCatalog' >> beam.io.ReadFromText('gs://sevir-306302/Catalog/CATALOG.csv', skip_header_lines =1)
       | 'SplitCatalog' >> beam.Map(lambda x: x.split(','))
       | 'format to dictCatalog' >> beam.Map(lambda x: {"id": x[0], "file_name": x[1], "file_index": x[2], 
                                                 "img_type": x[3],"time_utc": x[4], "minute_offsets": x[5], "episode_id": x[6], 
                                                 "event_id": x[7], "event_type": x[8], "llcrnrlat": x[9], "llcrnrlon": x[10], 
                                                 "urcrnrlat": x[11],"urcrnrlon": x[12], "proj": x[13], "size_x": x[14], 
                                                 "size_y": x[15], "height_m": x[16], "width_m": x[17],"data_min": x[18], 
                                                 "data_max": x[19], "pct_missing": x[20]})
                                     
      # | 'DelIncompleteData' >> beam.Filter(discard_incomplete)
       | 'ConvertypesCatalog' >> beam.Map(convert_types)
       | 'DelUnwantedDataCatalog' >> beam.Map(dropcolums)

       | 'WriteToBigQuery' >> beam.io.WriteToBigQuery(
           '{0}:sevirdataset.catalogcsv'.format(PROJECT_ID),
           schema=SCHEMACATALOG,
           write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND))


(p | 'ReadDataStorm' >> beam.io.ReadFromText('gs://sevir-306302/Storm/StormEvents_details_new.csv', skip_header_lines =1)
       | 'SplitStormData' >> beam.Map(lambda x: x.split(','))
       | 'format to dict Storm' >> beam.Map(lambda x: {"BEGIN_YEARMONTH": x[0], "BEGIN_DAY": x[1], "BEGIN_TIME": x[2], 
                                                 "END_YEARMONTH": x[3],"END_DAY": x[4], "END_TIME": x[5], "EPISODE_ID": x[6], 
                                                 "EVENT_ID": x[7], "STATE": x[8], "STATE_FIPS": x[9], "YEAR": x[10], 
                                                 "MONTH_NAME": x[11],"EVENT_TYPE": x[12], "CZ_TYPE": x[13], "CZ_FIPS": x[14], 
                                                 "CZ_NAME": x[15], "WFO": x[16], "BEGIN_DATE_TIME": x[17],"CZ_TIMEZONE": x[18], 
                                                 "END_DATE_TIME": x[19], "INJURIES_DIRECT": x[20],
                                                 "INJURIES_INDIRECT": x[21], "DEATHS_DIRECT": x[22], "DEATHS_INDIRECT": x[23],"DAMAGE_PROPERTY": x[24],
                                                 "DAMAGE_CROPS": x[25], "SOURCE": x[26], "MAGNITUDE": x[27],"MAGNITUDE_TYPE": x[28],
                                                 "FLOOD_CAUSE": x[29], "CATEGORY": x[30], "TOR_F_SCALE": x[31],"TOR_LENGTH": x[32],
                                                 "TOR_WIDTH": x[33], "TOR_OTHER_WFO": x[34], "TOR_OTHER_CZ_STATE": x[35],"TOR_OTHER_CZ_FIPS": x[36],
                                                 "TOR_OTHER_CZ_NAME": x[37], "BEGIN_RANGE": x[38], "BEGIN_AZIMUTH": x[39],"BEGIN_LOCATION": x[40],
                                                 "END_RANGE": x[41], "END_AZIMUTH": x[42], "END_LOCATION": x[43],"BEGIN_LAT": x[44],
                                                 "BEGIN_LON": x[45], "END_LAT": x[46], "END_LON": x[47],"EPISODE_NARRATIVE": x[48],
                                                 "EVENT_NARRATIVE": x[49], "DATA_SOURCE": x[50]})
                                     
      # | 'DelIncompleteData' >> beam.Filter(discard_incomplete)
      # | 'Convertypes_storm' >> beam.Map(convert_types_storm)
       | 'DelUnwantedDataStorm' >> beam.Map(dropcolumsstorm)

       | 'WriteToBigQueryStorm' >> beam.io.WriteToBigQuery(
           '{0}:sevirdataset.storm_details'.format(PROJECT_ID),
           schema=SCHEMASTORM,
           write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND))
result = p.run()

,id,file_name,file_index,img_type,time_utc,minute_offsets,episode_id,event_id,event_type,llcrnrlat,...,urcrnrlat,urcrnrlon,proj,size_x,size_y,height_m,width_m,data_min,data_max,pct_missing
0,R18032505027684,vis/2018/SEVIR_VIS_RANDOMEVENTS_2018_0321_0331.h5,0,vis,2018-03-25 05:00:00,-120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...,NaN,NaN,NaN,33.216708,...,36.336627,-87.070254,+proj=laea +lat_0=38 +lon_0=-98 +units=m +a=63...,768,768,384000.0,384000.0,-0.003361,0.005600,0.0
1,R18032505027677,vis/2018/SEVIR_VIS_RANDOMEVENTS_2018_0321_0331.h5,1,vis,2018-03-25 05:00:00,-120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...,NaN,NaN,NaN,33.084309,...,36.213723,-87.301535,+proj=laea +lat_0=38 +lon_0=-98 +units=m +a=63...,768,768,384000.0,384000.0,-0.003361,0.005600,0.0
2,R18032505027721,vis/2018/SEVIR_VIS_RANDOMEVENTS_2018_0321_0331.h5,2,vis,2018-03-25 05:00:00,-120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...,NaN,NaN,NaN,46.661866,...,50.883159,-120.009277,+proj=laea +lat_0=38 +lon_0=-98 +units=m +a=63...,768,768,384000.0,384000.0,-0.002870,0.005548,0.0
3,R18032516508249,vis/2018/SEVIR_VIS_RANDOMEVENTS_2018_0321_0331.h5,3,vis,2018-03-25 16:50:00,-120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...,NaN,NaN,NaN,40.883237,...,43.686191,-79.903987,+proj=laea +lat_0=38 +lon_0=-98 +units=m +a=63...,768,768,384000.0,384000.0,0.029911,0.785600,0.0
4,R18032516507621,vis/2018/SEVIR_VIS_RANDOMEVENTS_2018_0321_0331.h5,4,vis,2018-03-25 16:50:00,-120:-115:-110:-105:-100:-95:-90:-85:-80:-75:-...,NaN,NaN,NaN,44.946047,...,49.169436,-120.575175,+proj=laea +lat_0=38 +lon_0=-98 +units=m +a=63...,768,768,384000.0,384000.0,0.028560,0.792610,0.0


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201905,9,1554,201905,9,1830,137295,824116,TEXAS,48,...,7.0,NNE,SAN GERONIMO,29.7898,-98.6406,29.7158,-98.7744,Thunderstorms developed along a cold front as ...,Thunderstorms produced heavy rain that led to ...,CSV
1,201907,15,1640,201907,15,1641,140217,843354,MINNESOTA,27,...,5.0,E,ROCK CREEK,45.7700,-92.8700,45.7700,-92.8700,An area of low pressure over southern Manitoba...,A 14-inch tree fell due to the winds and damag...,CSV
2,201909,4,1229,201909,4,1229,141212,848333,VERMONT,50,...,1.0,W,LUDLOW,43.4000,-72.7200,43.4000,-72.7200,A strong mid-level disturbance and cold front ...,Quarter size hail reported at Okemo Ski resort.,CSV
3,201909,26,1554,201909,26,1554,141215,848338,NEW YORK,36,...,5.0,SW,MADRID,44.6988,-75.2220,44.6988,-75.2220,A strong mid-level disturbance and cold front ...,A few trees and power lines downed by thunders...,CSV
4,201902,19,2226,201902,19,2350,134941,808922,ARKANSAS,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rain was heavy at times on the 19th, and there...",One-quarter inch of freezing rain was measured...,CSV


,id,file_name,file_index,img_type,time_utc,minute_offsets,episode_id,event_id,event_type,llcrnrlat,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,S853086,vis/2019/SEVIR_VIS_STORMEVENTS_2019_0801_0831.h5,0,vis,2019-08-08 22:13:00,-119:-114:-109:-104:-99:-94:-89:-84:-79:-74:-6...,140872.0,853086,Flood,32.982201,...,0.0,ENE,HEREFORD,34.8147,-102.393,34.8236,-102.3926,Isolated thunderstorms developed during the af...,A Broadcast meteorologist reported localized s...,CSV
1,S853086,ir107/2019/SEVIR_IR107_STORMEVENTS_2019_0701_1...,357,ir107,2019-08-08 22:13:00,-119:-114:-108:-104:-99:-94:-89:-84:-78:-74:-6...,140872.0,853086,Flood,32.982201,...,0.0,ENE,HEREFORD,34.8147,-102.393,34.8236,-102.3926,Isolated thunderstorms developed during the af...,A Broadcast meteorologist reported localized s...,CSV
2,S853086,ir069/2019/SEVIR_IR069_STORMEVENTS_2019_0701_1...,357,ir069,2019-08-08 22:13:00,-119:-114:-109:-104:-99:-94:-89:-84:-79:-74:-6...,140872.0,853086,Flood,32.982201,...,0.0,ENE,HEREFORD,34.8147,-102.393,34.8236,-102.3926,Isolated thunderstorms developed during the af...,A Broadcast meteorologist reported localized s...,CSV
3,S853086,vil/2019/SEVIR_VIL_STORMEVENTS_2019_0701_1231.h5,269,vil,2019-08-08 22:13:00,-118:-113:-108:-103:-98:-93:-88:-83:-78:-73:-6...,140872.0,853086,Flood,32.982201,...,0.0,ENE,HEREFORD,34.8147,-102.393,34.8236,-102.3926,Isolated thunderstorms developed during the af...,A Broadcast meteorologist reported localized s...,CSV
4,S853086,lght/2019/SEVIR_LGHT_ALLEVENTS_2019_0801_0901.h5,0,lght,2019-08-08 22:13:00,NaN,140872.0,853086,Flood,32.982201,...,0.0,ENE,HEREFORD,34.8147,-102.393,34.8236,-102.3926,Isolated thunderstorms developed during the af...,A Broadcast meteorologist reported localized s...,CSV


  Using cached https://files.pythonhosted.org/packages/ad/fc/6e8c449185cb8862af353c1164100ff75e32d55ba1de3baf9eaa01b7d2a9/google_cloud_core-1.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/52/4b086e0d15245b648e2b6d408fb2f2974a3a5a405de5d7fae25cd085a3fa/google_resumable_media-1.2.0-py2.py3-none-any.whl
  Found existing installation: google-cloud-core 0.28.1
    Uninstalling google-cloud-core-0.28.1:
      Successfully uninstalled google-cloud-core-0.28.1
  Found existing installation: google-resumable-media 0.5.1
    Uninstalling google-resumable-media-0.5.1:
      Successfully uninstalled google-resumable-media-0.5.1


google-cloud-bigquery 0.23.0 has requirement google-cloud-core<0.24dev,>=0.23.1, but you'll have google-cloud-core 1.6.0 which is incompatible.
google-cloud-core 1.6.0 has requirement google-auth<2.0dev,>=1.24.0, but you'll have google-auth 1.21.1 which is incompatible.
google-cloud-core 1.6.0 has requirement six>=1.12.0, but you'll have six 1.10.0 which is incompatible.


DefaultCredentialsError: File C:/Users/gnana/PycharmProjects/Assignment/sevir/sevir-306302-fdeea5226bfa.json was not found.